# Análisis de sentimientos(Azure Text Analytics)

In [ ]:
import pandas as pd 
import time

In [ ]:
# Instaalmos libreria 
#! pip install azure-ai-textanalytics --pre

In [ ]:
# Definimos las variables de conexion.Estas las tenemso que sacar de nuestro portal de Azure.

key = "Rellenar con claves personales"
endpoint = "Rellenar con claves personales"

In [ ]:
#Cree una función para crear una instancia del objeto TextAnalyticsClient con los objetos key y endpoint creados anteriormente. Luego, cree un cliente.

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [ ]:
#Importamos csv que quremso Analizar.En este caso utilizaremso el descargado anteriormente.
df = pd.read_csv('LaIslaDeLastenataciones5-tweets-limpio-completo.csv')

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
#Pasamos la columna de Text para analizar.

df['Text']

In [ ]:
#Creamos función para llamar a la API.

def sentiment_analysis_example(client, tweet):

    documents = [tweet]
    response = client.analyze_sentiment(documents=documents)[0]
    return response


In [ ]:
#Creamos funcion para iterar todos los tweet y crear DF.

tweets_df=[] 
sentence_df=[] 

#Con el primer for sacaremos el sentimiento del tweet al completo
for i in range(df['Text'].count()):
    try:
        tweet = df['Text'][i]
        response = sentiment_analysis_example(client, tweet)
    ''' print("Document Sentiment: {}".format(response.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            response.confidence_scores.positive,
            response.confidence_scores.neutral,
            response.confidence_scores.negative,
        ))'''
        
        # Atributos del Tweet con los que rellenaremos el primr DF.
        tweets_df.append([tweet ,response.sentiment,response.confidence_scores.positive, response.confidence_scores.neutral,response.confidence_scores.negative])
        #print(tweets_df)

        #Con este segundo for sacamos el sentimiento por frases.
        for idx, sentence in enumerate(response.sentences):
            '''print("Sentence: {}".format(sentence.text))
            print("Sentence {} sentiment: {}".format(idx+1, sentence.sentiment))
            print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))'''
            
            #Atributos de la frase  con las que rellenaremos segundo DF.
            sentence_df.append([sentence.text,sentence.sentiment,sentence.confidence_scores.positive, sentence.confidence_scores.neutral,sentence.confidence_scores.negative])
            #print(sentence_df)


        print('Vamos a comprobar si tiene que esperar',i)
        
        # Cremos DF
        tweets_df_final =pd.DataFrame(tweets_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])         
        sentence_df_final =pd.DataFrame(sentence_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])

        #Guardamos DF
        tweets_df_final.to_csv('Sentimiento-tweets-5.csv', sep=',', index = False)
        sentence_df_final.to_csv('Sentimientos-frases-5.csv', sep=',', index = False)
            #comprobar limite
        time.sleep(0.25)

            #time.sleep(0.25) #Limite 
    except Exception:
        print("Error en la llamada:" + str(i))
        pass

#Podemos sacarlo fuera, pero si falla durante el proceso pierdo toda la información
#tweets_df_final =pd.DataFrame(tweets_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])         
#sentence_df_final =pd.DataFrame(sentence_df,columns=['Text','Total sentimiento','Positivo','Neutral','Negativo'])


#tweets_df_final.to_csv('Sentimiento-tweets-5.csv', sep=',', index = False)
#sentence_df_final.to_csv('Sentimientos-frases-5.csv', sep=',', index = False)

In [ ]:
#Importamso csv que hemos obtenido 
import pandas as pd 
df1 = pd.read_csv('Sentimiento-tweets-5.csv')
df2 =pd.read_csv('Sentimientos-frases-5.csv')

In [ ]:
df1['Total sentimiento'].value_counts()

In [ ]:
df1.head()

In [ ]:

df1.rename({ 'Text': 'Text_1'}, axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
#Concatenamos df para tener toda la informacióm
df_concat =pd.concat([df,df1],axis =1) 

In [ ]:
df_concat.info()

In [ ]:
df_concat.drop(['Unnamed: 0','Text_1',],1, inplace = True)

In [ ]:
#Guardamos documento en la versión que queramos.
df_concat.to_excel('DF_completo_final_IT5.xlsx')